# Programa Simplex

In [2]:
import numpy as np
import pandas as pd

#Simplex normal: 
Es decir este funciona en los casos donde no es necesario usar variables artificiales.

In [3]:
class Simplex:
    numeroVariables=None
    numeroRestricciones=None
    funcionObjetivo=[None]
    ladoDerecho=[None]
    relacion=[None]
    matrizRestricciones=None
    
    #tableau=generaTableau()

    def __init__(self,numeroVariables,numeroRestricciones,funcionObjetivo,ladoDerecho,relacion,matrizRestricciones):
      self.numeroVariables=numeroVariables
      self.numeroRestricciones=numeroRestricciones
      self.funcionObjetivo=funcionObjetivo
      self.ladoDerecho=ladoDerecho
      self.relacion=relacion
      self.matrizRestricciones=matrizRestricciones
      self.generaTableau()

    def generaTableau(self):
      matrizAux= [ [0 for columna in range (self.numeroRestricciones)] for renglon in range (self.numeroRestricciones) ] #Matriz para crear las variables de holgura y exceso
      variableArtificial=[] #Guarda las posiciones de las variables artificales
        
      #variables de holgura y exceso   
      if ( 2 in self.relacion or 3 in self.relacion ):
        for tipoRelacion in range (self.numeroRestricciones): 
          if ( self.relacion[tipoRelacion] == 2 ):   
            matrizAux[tipoRelacion][tipoRelacion] = 1
          else: 
            matrizAux[tipoRelacion][tipoRelacion] = -1
            variableArtificial.append(tipoRelacion) #Guarda la posición de la variable artificial

      #Variables artificiales  
      if ( len(variableArtificial) != 0 ):  #Si hay variables de exceso añadimos las varibles artificiales
        matrizArtificial = [[0 for columna in range(len(variableArtificial))] for renglon in range(self.numeroRestricciones) ]
        for artificial in range (len(variableArtificial)):
          matrizArtificial[variableArtificial[artificial]][artificial] =1

      #---------------------------------- UNIÓN ----------------------------------
      #Une las matrices de H y E, variables artificiales y Lado derecho 
      
      def uneMatriz(matriz,columnas):
        for renglon in range (self.numeroRestricciones):
          for columna in range (columnas):
            self.matrizRestricciones[renglon].append(matriz[renglon][columna])

      #Añade matriz de holgura y exceso
      uneMatriz(matrizAux,self.numeroRestricciones)
      if (len(variableArtificial) != 0): 
      #Añade matriz de variables artificiales
        uneMatriz(matrizArtificial,len(variableArtificial))
      #Añade Lado derecho
      for renglon in range (self.numeroRestricciones):
        self.matrizRestricciones[renglon].append(self.ladoDerecho[renglon])
      #Agrega la función objetivo al tableau y agrea ceros en los espacios necesarios     
      #for columna in range ((len(self.matrizRestricciones[0]) - self.numeroVariables)):
       # self.funcionObjetivo.append(0)
      self.funcionObjetivo += [0 for _ in range (len(self.matrizRestricciones[0]) - self.numeroVariables)]
      self.matrizRestricciones.insert(0,self.funcionObjetivo)   
      #Matriz
      self.matrizRestricciones=np.array(self.matrizRestricciones)
      return self.matrizRestricciones
    
    def cuentaVariablesArtificiales(self): #Regresa el numero de variables artificiales
      return (len(self.matrizRestricciones[0])-self.numeroRestricciones-self.numeroVariables-1)
    
    def variablesBasicas(self,basicas): #Regresa las variables que están dentro de la base y regresa una lista con los índices.
      for vectorColumna in range (len(self.matrizRestricciones[0])-1): #ignora el vector LD 
        if ( ( (np.count_nonzero(self.matrizRestricciones[1:,vectorColumna] == 0) == self.numeroRestricciones-1) and (np.count_nonzero(self.matrizRestricciones[1:,vectorColumna] == 1) == 1 ) ) ): 
          basicas.append(vectorColumna)
      return basicas

    def imprime(self): #Imprime el tableau
      basicas=[]
      nombreColumnas =[]
      indexBase = []
      self.variablesBasicas(basicas) #Obtiene la base

      for basica in (basicas):
        indexBase.insert(basica,"x"+str(basica+1))
      indexBase.insert(0,"Z")

      for columna in range(len(self.matrizRestricciones[0])-1):
        nombreColumnas.append("x"+str(columna+1))
      nombreColumnas.append("LD")    
      
      print(pd.DataFrame(self.matrizRestricciones,columns=nombreColumnas,index=indexBase))

    def infactible(self):
      base=[]
      self.variablesBasicas(base)
      base = np.array(base)
      if (self.cuentaVariablesArtificiales() == 0):
        return "No hay variables artificiales en la base"
      else:
        if( (np.count_nonzero( base > (self.numeroRestricciones + self.numeroVariables -1) )) != 0 ):
          return "Solución infactible"
          
    def Acotada(self, pivote):
      # No acotada
      indice = 0
      for i in range(len(self.matrizRestricciones[0])-1):
        if pivote == self.matrizRestricciones[0][i]:
          indice = i  # posicion de la columna del pivote

          break;

      Columna = self.matrizRestricciones[:, indice]

      np.array(Columna)
      Columna = np.delete(Columna,0)
      
      noacot = 0
      # print(Columna)
      i = 1
      for i in range(len(Columna)):
        if (Columna[i] <= 0):
          noacot += 1  # Numero de elementos de la columna pivote menores o iguales que cero

      if noacot == len(Columna):
        return False
      else:
        return True


    def Pivote(self): #Encuentra el pivote
      lista = []
      pivote=0
      for i in range(len(self.matrizRestricciones[0]) - 1):
        lista.append(self.matrizRestricciones[0][i])
      np.array(lista)

      pivote=np.amax(lista)

      return pivote

    def Optimalidad(self):
      pivote = self.Pivote()

      if pivote > 0:
        return True
      else:
        optimo = self.matrizRestricciones[0][len(self.matrizRestricciones[0]) - 1]
        print("ÓPTIMO: " + str(optimo) + "\n")
        self.infactible()
        return False


    # ----------------------------SOLUCION SIMPLEX------------------

    def solucion(self):

      def pivoteo(matriz, listaCol, listRen):
        mat1 = []
        for i in range(len(listaCol)):
          mat1.append([])
          for j in range(len(matriz[0])):
            mat1[i].append(matriz[i][j] - listRen[j] * listaCol[i])

        matriz = np.array(mat1)

        return matriz

      def encuentrapivote(matriz,pivote):
        indice = 0
        for i in range(len(self.matrizRestricciones[0])-1):
          if pivote == matriz[0][i]:
            indice = i  # posicion de la columna del pivote
            break;

        Columna = matriz[:, indice]  # Columna pivote
        division = matriz[:, len(matriz[0]) - 1] / Columna  # Division LD / columna pivote

        division1 = []
        for i in range(len(division)):
          if division[i] > 0:
            division1.append(division[i])

        division1 = np.array(division1)
        menor = np.amin(division1)  # obtenemos numero mas pequeño

        posicionpivote = np.where(np.amin(division1) == division)  # posicion de la fila del pivote
        pivote = matriz[posicionpivote, indice]  # posicion del pivote
        Renglonpiv = matriz[posicionpivote][0]  # Renglon pivote

        Renglonpiv = np.array(Renglonpiv[:] / pivote[0][0])

        # Renglonpiv = np.array(aux)
        matriz = pivoteo(matriz, Columna, Renglonpiv)

        matriz[posicionpivote[0][0]] = Renglonpiv

        return matriz

      self.imprime()
      i = 0

      # Itera el mientras no sea óptimo
      while self.Optimalidad():
        pivote = self.Pivote()
        if self.Acotada(pivote):
          print("Iteración " + str(i + 1))
          self.matrizRestricciones= encuentrapivote(self.matrizRestricciones,pivote)
          self.imprime()
          i += 1
        else:
          print("No acotada")
          break;

# Simplex método de big M:

Este ya una estensión la cual permite solucionar problemas que tengan variables artificiales utilizando el método Big M

In [4]:
class BigM(Simplex):
  M=None
  def __init__(self,numeroVariables,numeroRestricciones,funcionObjetivo,ladoDerecho,relacion,matrizRestricciones,M):
      Simplex.__init__(self,numeroVariables,numeroRestricciones,funcionObjetivo,ladoDerecho,relacion,matrizRestricciones)
      self.M=M
      self.tableauDF()
  
  def tableauDF(self):
    #numArtificial es la variable que nos indica la cantidad de variables artificiales que se agregarón 
    numArtificial =  self.cuentaVariablesArtificiales()
    
    #Agrega M
    for columna in range(numArtificial):
      self.matrizRestricciones[0][self.numeroRestricciones+self.numeroVariables + columna]= -1*self.M
      
    #Se hace cero los coeficientes de M
    self.matrizRestricciones = np.array(self.matrizRestricciones)
    for columna in range (len(self.matrizRestricciones[0])):
      self.matrizRestricciones[0,columna] = (sum(self.matrizRestricciones[:,columna])-self.matrizRestricciones[0,columna])*self.M + self.matrizRestricciones[0,columna]

# Ejemplo

In [5]:
numVar = 3
numRes = 4
FO = [50,20,25]
LD = [500, 350, 150,20]
relacion=[2,2,2,2]                     # 1 -> =, | 2 -> <=, 3 -> => | En este caso todas las variables son menores o iguales.
matRes = [[9,3,5],[5,4,0],[3,0,2],[0,0,1]]
M=10000
prueba = Simplex(numVar,numRes,FO,LD,relacion,matRes)
prueba.solucion()


    x1  x2  x3  x4  x5  x6  x7   LD
Z   50  20  25   0   0   0   0    0
x4   9   3   5   1   0   0   0  500
x5   5   4   0   0   1   0   0  350
x6   3   0   2   0   0   1   0  150
x7   0   0   1   0   0   0   1   20
Iteración 1
     x1    x2        x3   x4   x5         x6   x7      LD
Z   0.0  20.0 -8.333333  0.0  0.0 -16.666667  0.0 -2500.0
x1  0.0   3.0 -1.000000  1.0  0.0  -3.000000  0.0    50.0
x4  0.0   4.0 -3.333333  0.0  1.0  -1.666667  0.0   100.0
x5  1.0   0.0  0.666667  0.0  0.0   0.333333  0.0    50.0
x7  0.0   0.0  1.000000  0.0  0.0   0.000000  1.0    20.0
Iteración 2
     x1   x2        x3        x4   x5        x6   x7           LD
Z   0.0  0.0 -1.666667 -6.666667  0.0  3.333333  0.0 -2833.333333
x1  0.0  1.0 -0.333333  0.333333  0.0 -1.000000  0.0    16.666667
x2  0.0  0.0 -2.000000 -1.333333  1.0  2.333333  0.0    33.333333
x5  1.0  0.0  0.666667  0.000000  0.0  0.333333  0.0    50.000000
x7  0.0  0.0  1.000000  0.000000  0.0  0.000000  1.0    20.000000
Iteración 3
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:172: RuntimeWarning: divide by zero encountered in true_divide
